In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
today = '20220711'
input_year = '2022'
input_month = '07'
input_day = '05'
stock_code = '00020'

In [3]:
payload = ('__EVENTTARGET=btnSearch&__EVENTARGUMENT=&__VIEWSTATE=%2FwEPDwUKMTY0ODYwNTA0OWRkVb0Z6%2FtwMzkjq6As1zABy65ev%2BE%3D&__VIEWSTATEGENERATOR=A7B2BBE2&today='
+ today
+ '&sortBy=shareholding&sortDirection=desc&alertMsg=&txtShareholdingDate='
+ input_year
+ '%2F'
+ input_month
+ '%2F'
+ input_day
+ '&txtStockCode='
+ stock_code
+ '&txtStockName=&txtParticipantID=&txtParticipantName=&txtSelPartID=')

payload

'__EVENTTARGET=btnSearch&__EVENTARGUMENT=&__VIEWSTATE=%2FwEPDwUKMTY0ODYwNTA0OWRkVb0Z6%2FtwMzkjq6As1zABy65ev%2BE%3D&__VIEWSTATEGENERATOR=A7B2BBE2&today=20220711&sortBy=shareholding&sortDirection=desc&alertMsg=&txtShareholdingDate=2022%2F07%2F05&txtStockCode=00020&txtStockName=&txtParticipantID=&txtParticipantName=&txtSelPartID='

In [4]:
import requests
 
url = "https://www3.hkexnews.hk/sdw/search/searchsdw.aspx"
    
headers = {
    'authority': 'www3.hkexnews.hk',
    'content-type': 'application/x-www-form-urlencoded',
    'referer': 'https://www3.hkexnews.hk/sdw/search/searchsdw.aspx',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}


response = requests.request("POST", url, headers=headers, data=payload)
 
with open('resp.html', 'w') as f:
    f.write(response.text)

In [5]:
response

<Response [200]>

In [6]:
soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
soup


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html class="news-hkex eng" xmlns="http://www.w3.org/1999/xhtml">
<head><title>
	CCASS Shareholding Search
</title><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="/sdw/search/searchsdw.aspx" property="manualPath"/><meta content="A search engine to find shareholding information of CCASS Participants in the past of 12 months." property="description"/><meta content="CCASS Participant shareholding, participant shareholding, shareholding search, stock shareholding information" property="keywords"/><link href="/ncms/images/favicon.ico" rel="icon"/><link href="/ncms/css/main.css" rel="stylesheet" type="text/css"/><link href="css/sdw.css?v=1640057418154" rel="stylesheet" type="text/css"/>
<script src="/ncms/script/hkex_app.js" type="text/javascript"></script>
<

In [8]:
total_issued = int(soup.select("div.ccass-search-remarks div.summary-value")[0].text.replace(',', ''))
total_issued

33507400000

In [9]:
tbody = soup.select("tbody")[0]
tr_all = tbody.select("tr")

In [10]:
broker_code = []
broker_name = []
broker_addres = []
stock_holding = []
stock_holding_percent = []
for tr in tr_all:
    broker_code.append(tr.select("td")[0].text.split('\n')[2])
    broker_name.append(tr.select("td")[1].text.split('\n')[2])
    broker_addres.append(tr.select("td")[2].text.split('\n')[2])
    stock_holding.append(int(tr.select("td")[3].text.split('\n')[2].replace(',', '')))
    stock_holding_percent.append(float(tr.select("td")[4].text.split('\n')[2].replace('%', '')))
data_dict = {'Broker_code': broker_code,
            'Broker_name': broker_name,
            'Stock_holding': stock_holding,
            'Stock_holding_percent': stock_holding_percent}
df = pd.DataFrame.from_dict(data_dict)
df

,Broker_code,Broker_name,Stock_holding,Stock_holding_percent
0,B01955,FUTU SECURITIES INTERNATIONAL,5522021872,16.48
1,B01654,CHINA INTERNATIONAL CAPITAL CORPORATION,3630717856,10.83
2,B01829,HUATAI FINANCIAL HOLDINGS (HONG KONG),1305285159,3.89
3,B01565,GUOTAI JUNAN SECURITIES (HONG KONG) LTD,1052545101,3.14
4,B01143,HAITONG INTERNATIONAL SECURITIES CO LTD,1003911266,2.99
...,...,...,...,...
395,B01832,MIZUHO SECURITIES ASIA LTD,1000,0.00
396,B01935,MAGIC COMPASS SECURITIES LTD,1000,0.00
397,B01950,GLOBAL GROUP SECURITIES LTD,1000,0.00
398,B02155,ADEN FINANCIAL GROUP LTD,1000,0.00


In [11]:
df['Stock_holding_percent'].sum()

57.15999999999999

In [12]:
df[df.Broker_code == 'C00093'].Stock_holding / total_issued

59    0.000204
Name: Stock_holding, dtype: float64

In [13]:
df.query('Broker_code == "B01389"')

,Broker_code,Broker_name,Stock_holding,Stock_holding_percent
90,B01389,ZHONGRONG PT SECURITIES LTD,1495000,0.0


In [14]:
df['Stock_holding'].sum()

19382288428

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Broker_code            400 non-null    object 
 1   Broker_name            400 non-null    object 
 2   Stock_holding          400 non-null    int64  
 3   Stock_holding_percent  400 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 12.6+ KB


In [16]:
date = soup.select("input#txtShareholdingDate")
date[0]['value']

'2022/07/05'

In [17]:
df.to_excel('0020HK.xlsx')